In [8]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
import pandas as pd
from dotenv import load_dotenv

load_dotenv()  

# dan jurafsky's string2string library
from string2string.similarity import JaroSimilarity


from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate, LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

from prompting.prompts import DataTemplates

In [33]:
df = pd.read_csv("../abstracts_final.csv")

In [ ]:
template = ChatPromptTemplate.from_messages(DataTemplates().medicine_abstract_prompt())

In [40]:
def generate_row(text: str, temperature) -> str:
    llm = ChatOpenAI(temperature=temperature)
    chat = LLMChain(llm=llm, prompt=template)
    output = chat.run({"text": text})
    return output

In [51]:
temperatures = [0.3, 0.5, 0.7, 0.9, 1]

In [52]:
generations = []
for t in temperatures:
    for i, row in df.iterrows():
        text = row["texts"]
        generations.append({"temperature": t, "text": generate_row(text)})


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID a00b1a6d133e74641c9e694f3461860d in your message.).


In [58]:
df2 = pd.DataFrame(generations).reset_index()
df2["intex"] = df2["index"].apply(lambda x: x % 16)

In [64]:
df[temperatures] = df2.pivot(index="intex", columns="temperature", values="text")

In [66]:
df.to_csv("../data/generated_outputs.csv", index=False)

### We need to determine how likely the texts are to be generated by ChatGPT.
We will generate 10 texts for each abstract. We will then compare the similarity between the real human answers and GPT-answers. 

1. Select 3 generations and measure the average similarity to the other ChatGPT answers
2. Select the real texts and see how similar they are to the generations.
3. Use the MTurk data and measure how similar the texts are. 

In [50]:

# Create an instance of the JaroSimilarity class
jaro_similarity = JaroSimilarity()

# Compute the Jaro similarity scores of the following pairs of words
score = jaro_similarity.compute('FARMVILLE', 'FAREMVIEL')
print(f'The Jaro similarity score between FARMVILLE and FAREMVIEL is {score:.2f}')


The Jaro similarity score between FARMVILLE and FAREMVIEL is 0.88


In [30]:
from string2string.alignment import LongestCommonSubsequence

# Create an instance of the LongestCommonSubsequence class
lcsubsequence = LongestCommonSubsequence()

length, candidates = lcsubsequence.compute(
    str1="abcdefghijklmnopqrstuvwxyz",
    str2="xzytawbvckjfhgslrqponmedui", 
    returnCandidates=True
)

# Print the length of the longest common subsequence
print(f'The length of the longest common subsequence is {length}.')

# Print the candidates
print(f'The candidates are {candidates}.')